# Stacking Meta, TFIDF, and DocEmbedding Models

In [ ]:
pip install mlxtend

In [41]:
import pandas as pd
import numpy as np
import sklearn
from mlxtend import classifier
from mlxtend import feature_selection
from mlxtend.classifier import StackingClassifier
from mlxtend.feature_selection import ColumnSelector
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
import numpy as np
import pandas as pd
import en_core_web_sm
from wordcloud import WordCloud, STOPWORDS
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import string
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import re
import sklearn
from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
# import en_core_web_sm
# from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer


# Create our list of stopwords
nlp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()


## TOKENIZER

In [42]:
STOP_WORDS = stop_words.union({'th','st'})

In [43]:
#IMPORTS
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import string

punctuation = string.punctuation
punctuation = punctuation+"..."+"--"+"“"+"”"+"``"+"''"+"’"+"–"+"—"+"‘"
lemmatizer = WordNetLemmatizer()

In [44]:
contr_dict={"I’m": "I am",
            "won’t": "will not",
            "’s" : "", 
            "’ll":"will",
            "’ve ":"have ",
            "n’t":" not",
            "’re": "are",
            "’d": "would",
            "y’all": "all of you",
            "I'm": "I am",
            "won't": "will not",
            "'s" : "", 
            "'ll":"will",
            "'ve ":"have ",
            "n't":"not",
            "'re": "are",
            "'d": "would",
            "y'all": "all of you"}
contr_dict.keys()


dict_keys(['I’m', 'won’t', '’s', '’ll', '’ve ', 'n’t', '’re', '’d', 'y’all', "I'm", "won't", "'s", "'ll", "'ve ", "n't", "'re", "'d", "y'all"])

In [45]:
def replace_contractions(sentence, contr_dict=contr_dict):
    for contr in contr_dict.keys():
        if contr in sentence:
            sentence = sentence.replace(contr,contr_dict[contr])
    return sentence

In [46]:
import re 
  
def remove_numbers(tokens): 
    pattern = '[0-9]'
    tokens_updated = [re.sub(pattern, '', token) for token in tokens] 
    return tokens_updated

In [47]:
# function to convert nltk tag to wordnet tag
# this is important because having the POS tag improves lemmatization
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

In [48]:
#tokenization and lemmatization function
def tokenize_sentence(sentence):
    #remove contractions
    sentence = replace_contractions(sentence, contr_dict=contr_dict)
    
    #tokenize the sentence
    mytokens = nltk.word_tokenize(sentence)

    #remove numbers
    mytokens = remove_numbers(mytokens)
    
    #remove tokens left over that are only space char
    mytokens = [token for token in mytokens if len(token)>0]
    
    #tag tokens with part of speech
    nltk_tagged = nltk.pos_tag(mytokens)

    # remove punctuation
    nltk_tagged = [ word for word in nltk_tagged if word[0] not in punctuation ]
    
    #
    nltk_tagged = [word for word in nltk_tagged if word[0].isalpha()]
    
    # strip all tokens and make lowercase 
    nltk_tagged = [ (word[0].lower().strip(),word[1]) for word in nltk_tagged ]
    
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    
    lemmatized_tokens = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_tokens.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_tokens.append(lemmatizer.lemmatize(word, tag))
            
        lemmatized_tokens = [word for word in lemmatized_tokens if word not in STOP_WORDS]
    return lemmatized_tokens


## DATA FRAME SUMMARY

In [49]:
df = pd.read_csv('meta_features.csv')

In [50]:
df.head()

,article_id,propaganda,propaganda_type,text,prop_txt_snippet,sent_#,sentiment_score,abs_sent_score,punct_count,word_count,%adj,%verb,%adv,%noun,avg_word_length,strong_subjectives_count
0,701225819,non-propaganda,NaN,South Florida Muslim Leader Sofian Zakkout’s D...,NaN,1,0.0000,0.0000,0,9,0.000000,0.000000,0.000000,0.000000,5.444444,0
1,701225819,propaganda,"Name_Calling,Labeling","David Duke, the white supremacist icon and for...",Grand Wizard of the Ku Klux Klan,2,0.5423,0.5423,4,26,0.020548,0.006849,0.013699,0.006849,4.423077,2
2,701225819,propaganda,Loaded_Language,"However, one individual who represents the Mus...",enamored,3,0.3612,0.3612,4,27,0.017241,0.017241,0.005747,0.022989,5.000000,0
3,701225819,non-propaganda,NaN,"Last month, once again, Zakkout chose to showc...",NaN,4,0.0000,0.0000,5,22,0.021127,0.021127,0.014085,0.035211,5.045455,0
4,701225819,non-propaganda,NaN,The postings can be rivaled only by Zakkout’s ...,NaN,5,0.0000,0.0000,1,11,0.014493,0.043478,0.014493,0.028986,4.636364,0


In [51]:
stack_model_df = df.drop(['article_id','propaganda_type','prop_txt_snippet','sent_#'], axis=1)
stack_model_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15172 entries, 0 to 15171
Data columns (total 12 columns):
propaganda                  15172 non-null object
text                        15172 non-null object
sentiment_score             15172 non-null float64
abs_sent_score              15172 non-null float64
punct_count                 15172 non-null int64
word_count                  15172 non-null int64
%adj                        15172 non-null float64
%verb                       15172 non-null float64
%adv                        15172 non-null float64
%noun                       15172 non-null float64
avg_word_length             15172 non-null float64
strong_subjectives_count    15172 non-null int64
dtypes: float64(7), int64(3), object(2)
memory usage: 1.4+ MB


In [52]:
y = stack_model_df['propaganda']
X = stack_model_df.drop('propaganda', axis=1)

In [148]:
text_2 = X['text']

In [149]:
X['text_2']=text_2

In [150]:
X.columns

Index(['text', 'sentiment_score', 'abs_sent_score', 'punct_count',
       'word_count', '%adj', '%verb', '%adv', '%noun', 'avg_word_length',
       'strong_subjectives_count', 'embeds', 'text_2'],
      dtype='object')

In [61]:
y = [1 if label == 'propaganda' else 0 for label in y]

In [157]:
from sentence_transformers import SentenceTransformer
model_embeds = SentenceTransformer('bert-base-nli-mean-tokens')

In [71]:
X['embeds'] = model_embeds.encode(np.array(X['text']))

In [118]:
embeds_array = np.array(X['embeds'])

In [171]:
df3 = pd.DataFrame(X['embeds'].values.tolist())

In [173]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15172 entries, 0 to 15171
Columns: 768 entries, 0 to 767
dtypes: float64(768)
memory usage: 88.9 MB


In [187]:
X_embeds = X.join(df3)

In [188]:
X_embeds = X_embeds.drop(['embeds','text_2'], axis=1)

In [194]:
X_embeds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15172 entries, 0 to 15171
Columns: 779 entries, text to 767
dtypes: float64(775), int64(3), object(1)
memory usage: 90.2+ MB


In [221]:
just_embeds = X_embeds[list(X_embeds.columns[11:779])]

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.164120,-0.019730,0.203762,-0.149297,0.696591,-0.960957,0.206785,-0.540251,0.005157,0.196422,...,-0.075324,-0.620497,-0.303472,0.297969,1.019830,-0.523595,-0.525541,0.337973,0.422184,-0.103847
1,-0.107122,0.944672,-0.452394,-0.229796,-0.732791,-0.794121,1.180954,-0.231252,0.270980,0.060620,...,-0.069693,0.726365,-0.519457,-1.590454,0.438942,-0.663646,0.016005,-0.209250,-0.010170,0.619601
2,0.123370,0.182647,0.767236,-0.386340,0.239815,-1.276435,0.194789,-0.089553,0.052065,-0.120395,...,-0.506447,-1.584028,-0.943574,-1.807983,0.863276,-0.925124,-0.041773,0.223409,-0.194369,-0.310801
3,-0.212266,0.199948,1.093049,-0.079362,0.368825,-0.464936,0.845249,0.332470,-0.713169,-0.262536,...,-0.256164,-1.225924,-1.114888,-0.392965,0.123237,-0.763239,-0.119133,0.222245,0.562734,0.133977
4,-0.082211,0.736390,0.620443,0.304812,0.169626,-0.476543,0.072635,0.401900,-0.055084,0.132189,...,-0.188720,0.061769,0.530789,-1.138050,-0.434086,0.134580,0.068409,-0.174906,0.013871,0.111412
5,-0.504248,0.465273,0.330162,-0.282355,0.195496,-1.244920,0.398042,0.102070,-0.213497,0.194332,...,-0.177531,-0.336883,-0.527773,0.365958,0.612401,-0.599148,-0.148439,-0.568717,0.717854,0.218804
6,-0.228612,0.261782,-0.260801,-0.214646,0.547143,-0.754278,0.029804,-0.378770,-0.459770,-0.506010,...,-0.112834,-0.710977,-0.672821,0.589150,0.225079,-0.839100,0.156029,0.297599,1.139234,-0.182229
7,-0.090555,0.529084,0.356458,-0.081107,0.355607,-0.283721,0.824368,0.386737,-0.593540,-0.308293,...,-0.310062,-0.642304,-1.027118,0.195643,0.058157,-0.368076,-0.401911,0.302515,0.576539,0.144626
8,0.010008,0.962858,-0.633553,-0.451542,-0.093861,-1.025709,0.891725,0.094641,0.341399,0.166679,...,-0.415344,-0.358977,-1.049884,0.403800,0.391501,-0.288238,-0.054236,-0.175665,1.020589,0.036927
9,-0.131621,0.445831,-0.713069,-0.268536,-0.454286,-0.289466,0.878650,0.204928,-0.058836,0.309174,...,-0.374667,-0.532018,-0.597064,-0.810273,0.433057,-0.209578,0.001626,-0.561577,-0.110241,-0.092543


In [222]:
X_train, X_test, y_train, y_test = train_test_split(X_embeds, y, test_size=0.33, random_state=42)

## TRANFORMERS FOR PIPELINES

In [224]:
scaler = StandardScaler()
tfidf = TfidfVectorizer(tokenizer = tokenize_sentence, 
                               min_df=5, max_df=0.7)

In [223]:
just_embeds_train = X_train[list(X_train.columns[11:779])]
just_embeds_test = X_test[list(X_test.columns[11:779])]

In [94]:
X_train['tfidf']=tfidf.fit_transform(X_train['text'])
X_test['tfidf']=tfidf.transform(X_test['text'])

/Users/sashaepelbaum/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/sashaepelbaum/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [227]:
train_tfidf = pd.DataFrame(X_train['tfidf'].values.tolist())
test_tfidf = pd.DataFrame(X_test['tfidf'].values.tolist())

KeyError: 'tfidf'

In [228]:
from sklearn_pandas import DataFrameMapper
column_tuples_train = [
    ('text', tfidf),
    ('sentiment_score', None),
    ('abs_sent_score', None),
    ('punct_count', None),
    ('word_count', None),
    ('%adj', None),
    ('%verb', None),
    ('%adv', None), 
    ('%noun', None),
    ('avg_word_length', None),
    ('strong_subjectives_count', None)
]


mapper_1 = DataFrameMapper(column_tuples_train, df_out=True)


In [229]:
X_train_mapped = mapper_1.fit_transform(X_train)

In [230]:
X_test_mapped = mapper_1.transform(X_test)

In [231]:
X_train_all = X_train_mapped.join(just_embeds_train)

In [233]:
X_test_all = X_test_mapped.join(just_embeds_test)

In [234]:
X_train_all[['sentiment_score','abs_sent_score','punct_count',
        'word_count','%adj','%verb','%adv','%noun',
        'avg_word_length',
         'strong_subjectives_count']] = scaler.fit_transform(X_train_[['sentiment_score',
                                        'abs_sent_score','punct_count',
                                       'word_count','%adj','%verb','%adv','%noun',
                                       'avg_word_length','strong_subjectives_count']])
X_test_all[['sentiment_score','abs_sent_score','punct_count',
        'word_count','%adj','%verb','%adv','%noun',
        'avg_word_length','strong_subjectives_count']]=scaler.transform(X_test[[
                                        'sentiment_score','abs_sent_score','punct_count',
                                       'word_count','%adj','%verb','%adv','%noun',
                                       'avg_word_length','strong_subjectives_count']])

In [142]:
X_train_1.head()

,sentiment_score,abs_sent_score,punct_count,word_count,%adj,%verb,%adv,%noun,avg_word_length,strong_subjectives_count,embeds,tfidf
13001,0.378942,-0.584117,0.107380,1.787735,0.125383,0.155285,-0.298495,0.849950,-0.302474,-0.814549,"[-0.17333102, 0.8431093, 0.81538683, -0.283583...","(0, 3226)\t0.16293961526714112\n (0, 2811)\..."
13937,0.069278,-1.026921,-0.712079,-0.409388,0.314773,0.438876,-0.540039,0.792049,-0.648888,0.585855,"[0.0918742, -0.13311641, -0.01790656, 0.670809...","(0, 3226)\t0.16293961526714112\n (0, 2811)\..."
12521,0.069278,-1.026921,0.517110,-0.182099,-0.861440,0.552504,-0.080325,-0.273210,0.703670,0.585855,"[-0.16667514, -0.09049596, 0.25222436, 0.00745...","(0, 3226)\t0.16293961526714112\n (0, 2811)\..."
9363,0.069278,-1.026921,1.336569,0.045189,-0.277755,-0.448369,-0.540039,0.398204,0.460798,-0.814549,"[-0.31779048, 0.6162055, 1.2091062, 0.37545416...","(0, 3226)\t0.16293961526714112\n (0, 2811)\..."
14796,0.069278,-1.026921,-0.712079,-1.091253,-0.861440,0.830264,-0.540039,0.370299,-0.448944,-0.814549,"[0.52177453, 0.31714925, 0.9615935, 0.6633711,...","(0, 3226)\t0.16293961526714112\n (0, 2811)\..."


In [243]:
X_train_all.to_csv('X_train_all_transformed.csv')
X_test_all.to_csv('X_test_all_transformed.csv')
pd.DataFrame(y_train).to_csv('y_train.csv')
pd.DataFrame(y_test).to_csv('y_test.csv')

In [383]:
X_train_all[X_train_all.columns[0:3262]].head()

,text_abandon,text_abedin,text_abid,text_ability,text_able,text_abnormal,text_abortion,text_abraham,text_absolute,text_absolutely,...,text_york,text_yorker,text_youare,text_young,text_youth,text_zakkout,text_zarate,text_zero,text_zionist,text_zone
13001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13937,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12521,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.394301,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9363,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14796,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [384]:
X_train_all[X_train_all.columns[3262:3272]].head()

,sentiment_score,abs_sent_score,punct_count,word_count,%adj,%verb,%adv,%noun,avg_word_length,strong_subjectives_count
13001,0.378942,-0.584117,0.107380,1.787735,0.125383,0.155285,-0.298495,0.849950,-0.302474,-0.814549
13937,0.069278,-1.026921,-0.712079,-0.409388,0.314773,0.438876,-0.540039,0.792049,-0.648888,0.585855
12521,0.069278,-1.026921,0.517110,-0.182099,-0.861440,0.552504,-0.080325,-0.273210,0.703670,0.585855
9363,0.069278,-1.026921,1.336569,0.045189,-0.277755,-0.448369,-0.540039,0.398204,0.460798,-0.814549
14796,0.069278,-1.026921,-0.712079,-1.091253,-0.861440,0.830264,-0.540039,0.370299,-0.448944,-0.814549


In [385]:
X_train_all[X_train_all.columns[3272:4040]].head()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
13001,-0.173331,0.843109,0.815387,-0.283583,0.355310,-0.363973,1.265644,-0.608121,-0.231724,-0.349867,...,-0.251001,-1.381743,-0.772514,-0.686197,-0.173441,-0.560432,-0.213605,0.026848,-0.679761,0.458733
13937,0.091874,-0.133116,-0.017907,0.670810,0.950990,0.213923,0.284865,-0.094425,0.490145,0.336642,...,0.335247,-0.472793,-0.247906,-1.039255,-0.247866,-0.295682,0.041536,-0.012181,-0.719729,0.212103
12521,-0.166675,-0.090496,0.252224,0.007459,0.778086,0.036884,0.177676,0.318603,0.459911,0.622604,...,-0.215225,-0.969704,-0.959864,-0.327255,-0.499393,-0.895054,-0.787237,-0.379003,-0.772760,-0.057005
9363,-0.317790,0.616206,1.209106,0.375454,0.138737,0.572094,0.818017,-0.115805,0.259089,0.281828,...,0.351264,0.681804,0.360461,-1.479023,-0.932557,0.317828,0.135406,-0.529601,-0.902091,-0.090235
14796,0.521775,0.317149,0.961594,0.663371,-0.138175,-0.537674,0.436469,1.175056,0.449023,0.549446,...,-0.331839,0.055938,1.047838,-1.078855,-0.295900,-0.543152,-0.308911,-0.110881,0.041825,0.191739


## Best Hyper-Parameters Taken From Optimizing Each Model with Random Search

In [252]:
from sklearn.ensemble import GradientBoostingClassifier

In [312]:
log_reg_embeds = LogisticRegression(C = 3, penalty = 'l2', solver='newton-cg', 
                                    class_weight = 'balanced', max_iter = 1000)

log_reg_tfidf = LogisticRegression(penalty = 'l2', C = 29.763514416313132, 
                                   solver = 'lbfgs', class_weight = 'balanced', max_iter = 1000)

log_reg_meta = LogisticRegression(penalty = 'l2', C = 0.012742749857031334, 
                                   solver = 'sag', class_weight = 'balanced', max_iter = 1000)



In [435]:
# Create logistic regression
logistic = LogisticRegression()

hyperparam_grid_logistic = {'penalty' : ['l1', 'l2'],
    'C' : np.logspace(-4, 4, 20),
    'solver' : ['newton-cg', 'lbfgs', 'sag', 'saga'],
    'class_weight': 'balanced'}


In [446]:
# # Create randomized search 5-fold cross validation and 100 iterations
clf_log = RandomizedSearchCV(logistic, hyperparam_grid_logistic, random_state=1, n_iter=200, cv=5, 
                         verbose=True, n_jobs=-1, scoring = 'f1')

In [447]:
embeds_cols = [i for i in X_train_all.columns[3272:4040]]
tfidf_cols = [str(i) for i in X_train_all.columns[0:3262]]
meta_cols = [i for i in X_train_all.columns[3262:3272]]

In [448]:
from sklearn.pipeline import make_union, make_pipeline
from sklearn.preprocessing import FunctionTransformer

def get_embeds_cols(df):
    return df[embeds_cols]

def get_tfidf_cols(df):
    return df[tfidf_cols]

def get_meta_cols(df):
    return df[meta_cols]


In [449]:
pipe_embeds = make_pipeline(FunctionTransformer(get_embeds_cols, validate=False), log_reg_embeds)
pipe_tfidf = make_pipeline(FunctionTransformer(get_tfidf_cols, validate=False), log_reg_tfidf)
pipe_meta = make_pipeline(FunctionTransformer(get_meta_cols, validate=False), log_reg_meta)

In [450]:
sclf = StackingClassifier(estimators=[('pipe_embeds', pipe_embeds), ('pipe_tfidf', pipe_tfidf), 
                                      ('pipe_meta', pipe_meta)], final_estimator=clf_log,
                          cv=5,stack_method='decision_function', n_jobs=-1,verbose=True)

In [458]:
clf_2 = LogisticRegression(C = 3, penalty = 'l2', solver='newton-cg', 
                                    class_weight = 'balanced', max_iter = 1000)

In [459]:
sclf2 = StackingClassifier(estimators=[('pipe_embeds', pipe_embeds), ('pipe_tfidf', pipe_tfidf), 
                                      ('pipe_meta', pipe_meta)], final_estimator=clf_2,
                          cv=5,stack_method='decision_function', n_jobs=-1,verbose=True)

In [460]:
sclf2.fit(X_train_all, y_train)

StackingClassifier(cv=5,
                   estimators=[('pipe_embeds',
                                Pipeline(memory=None,
                                         steps=[('functiontransformer',
                                                 FunctionTransformer(accept_sparse=False,
                                                                     check_inverse=True,
                                                                     func=<function get_embeds_cols at 0x1a1a060d90>,
                                                                     inv_kw_args=None,
                                                                     inverse_func=None,
                                                                     kw_args=None,
                                                                     validate=False)),
                                                ('logisticregression',
                                                 LogisticRegression(C=3,
                

In [461]:
# Predict target vector
stacked_preds_2 = sclf2.predict(X_test_all)

In [462]:
# Print the confusion matrix
print(sklearn.metrics.confusion_matrix(y_test, stacked_preds_2))

# Print the precision and recall, among other metrics
print(sklearn.metrics.classification_report(y_test, stacked_preds_2, digits=3))
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_test, stacked_preds_2))

[[1294 2216]
 [ 127 1370]]
              precision    recall  f1-score   support

           0      0.911     0.369     0.525      3510
           1      0.382     0.915     0.539      1497

    accuracy                          0.532      5007
   macro avg      0.646     0.642     0.532      5007
weighted avg      0.753     0.532     0.529      5007

0.6419123146578056


In [451]:
best_stacked_model = sclf.fit(X_train_all, y_train)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/Users/sashaepelbaum/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    9.7s finished


In [452]:
sclf.final_estimator_.get_params()

{'cv': 5,
 'error_score': nan,
 'estimator__C': 1.0,
 'estimator__class_weight': None,
 'estimator__dual': False,
 'estimator__fit_intercept': True,
 'estimator__intercept_scaling': 1,
 'estimator__l1_ratio': None,
 'estimator__max_iter': 100,
 'estimator__multi_class': 'auto',
 'estimator__n_jobs': None,
 'estimator__penalty': 'l2',
 'estimator__random_state': None,
 'estimator__solver': 'lbfgs',
 'estimator__tol': 0.0001,
 'estimator__verbose': 0,
 'estimator__warm_start': False,
 'estimator': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=100,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 'iid': 'deprecated',
 'n_iter': 200,
 'n_jobs': -1,
 'param_distributions': {'penalty': ['l1', 'l2'],
  'C': array([1.00000000e-04, 2.63665090e-04, 6.95192796e-04, 1.8

In [453]:
# # View best hyperparameters
# print('Best Penalty:', sclf.final_estimator_.get_params()['penalty'])
# print('Best C:', sclf.final_estimator_.get_params()['C'])
# print('Best solver:', sclf.final_estimator_.get_params()['solver'])

In [454]:
# Predict target vector
stacked_preds = best_stacked_model.predict(X_test_all)

In [455]:
# Print the confusion matrix
print(sklearn.metrics.confusion_matrix(y_test, stacked_preds))

# Print the precision and recall, among other metrics
print(sklearn.metrics.classification_report(y_test, stacked_preds, digits=3))
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_test, stacked_preds))

[[2257 1253]
 [ 387 1110]]
              precision    recall  f1-score   support

           0      0.854     0.643     0.734      3510
           1      0.470     0.741     0.575      1497

    accuracy                          0.672      5007
   macro avg      0.662     0.692     0.654      5007
weighted avg      0.739     0.672     0.686      5007

0.6922514544759033


In [457]:
import pickle 
  
# Save the trained model as a pickle string. 
saved_stacked_model = pickle.dumps(best_stacked_model) 
  
# Load the pickled model 
stacked_from_pickle = pickle.loads(saved_stacked_model) 
  
# Use the loaded pickled model to make predictions 
stacked_from_pickle.predict(X_test_all) 

PicklingError: Can't pickle <function get_embeds_cols at 0x1a1a0609d8>: it's not the same object as __main__.get_embeds_cols

In [ ]:
stacked_preds.to_csv('stacked_preds.csv')

In [299]:
from sklearn.decomposition import TruncatedSVD

In [300]:
svd = TruncatedSVD(n_components=500, n_iter = 50)
x_train_svd = svd.fit_transform(X_train_all)
x_test_svd = svd.transform(X_test_all)
print('LSA output shape:', x_train_svd.shape)
explained_variance = svd.explained_variance_ratio_.sum()
print("Sum of explained variance ratio: %d%%" % (int(explained_variance * 100)))

LSA output shape: (10165, 500)
Sum of explained variance ratio: 98%


In [301]:
# Create logistic regression
logistic = LogisticRegression()

hyperparam_grid_logistic = {'penalty' : ['l1', 'l2'],
    'C' : np.logspace(-4, 4, 20),
    'solver' : ['newton-cg', 'lbfgs', 'sag', 'saga'],
    'class_weight': 'balanced'}

In [305]:
# Create randomized search 5-fold cross validation and 100 iterations
clf_log_all = RandomizedSearchCV(logistic, hyperparam_grid_logistic, random_state=1, n_iter=1000, cv=5, 
                         verbose=True, n_jobs=-1, scoring = 'roc_auc')

In [306]:
# Fit randomized search
best_model_log_all = clf_log_all.fit(x_train_svd, y_train)

Fitting 5 folds for each of 1000 candidates, totalling 5000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 11.2min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed: 23.7min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed: 36.5min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed: 52.1min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed: 72.8min
[Parallel(n_jobs=-1)]: Done 3192 tasks      | elapsed: 97.0min
[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed: 121.4min
[Parallel(n_jobs=-1)]: Done 4992 tasks      | elapsed: 153.0min
[Parallel(n_jobs=-1)]: Done 5000 out of 5000 | elapsed: 153.1min finished


In [307]:
# View best hyperparameters
print('Best Penalty:', best_model_log_all.best_estimator_.get_params()['penalty'])
print('Best C:', best_model_log_all.best_estimator_.get_params()['C'])
print('Best solver:', best_model_log_all.best_estimator_.get_params()['solver'])

Best Penalty: l2
Best C: 0.012742749857031334
Best solver: newton-cg


In [308]:
# Predict target vector
log_all_preds = best_model_log_all.predict(x_test_svd)

In [310]:
# Print the confusion matrix
print(sklearn.metrics.confusion_matrix(y_test, log_all_preds))

# Print the precision and recall, among other metrics
print(sklearn.metrics.classification_report(y_test, log_all_preds, digits=3))

roc_auc_score(y_test, log_all_preds)

[[ 949 2561]
 [ 101 1396]]
              precision    recall  f1-score   support

           0      0.904     0.270     0.416      3510
           1      0.353     0.933     0.512      1497

    accuracy                          0.468      5007
   macro avg      0.628     0.601     0.464      5007
weighted avg      0.739     0.468     0.445      5007



0.6014510502486454